In [1]:
# For larger models with more than 2gb memory
# !git clone https://github.com/zldrobit/yolov5.git  # clone
# %cd yolov5
# !git checkout fix-exceeds-2gb

# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# %pip install -qr requirements.txt  # install


from yolov5 import utils
# display = utils.notebook_init()  # checks

# Convert to ONNX
Select the model version and input size. Default: YOLOV6s (640x480)

In [7]:
import os
from datetime import datetime
import requests
import shutil
import json
import pickle
import torch

# Libraries for pre and post processsing
# from ultralytics.data.augment import LetterBox
# from ultralytics.engine.results import Results
# from ultralytics.utils import ops
# from ultralytics.utils.plotting import Annotator, colors

# import onnx_runtime related package
import onnxruntime as rt

import numpy as np
import cv2


class ModelHandler(object):
    """
    A YOLOV5 Model handler implementation.
    """
    def __init__(self):
        
        self.initialized = False

        # Parameters for inference
        self.model = None

        # Parameters for pre-processing
        self.imgsz = 640 # default value for this usecase. 
        self.stride = 32 # default value for this usecase( differs based on the model selected )
        
        # Parameters for post-processing
        self.conf = 0.25
        self.iou = 0.45
        self.max_det = 300
        self.classes = None
        self.agnostic = False
        self.labels = {0: 'somke', 1: 'fire'}
        
        self.path = '/home/raw-data/'
        
        # print(os.listdir("/opt/ml/model/"))

    def initialize(self, context):
        # os.path.join(model_dir,'model_onnx.onnx')
        self.initialized = True

        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='/home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.pt')
        
    def preprocess(self, request):
        image_url = request['body'].decode()

        ## Set up the image URL and filename
        self.path = self.path+image_url.split("/")[-1] if self.path == '/home/raw-data/' else '/home/raw-data/'+image_url.split("/")[-1]
        # self.path = self.path+self.filename if self.path=='' else

        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            # Open a local file with wb ( write binary ) permission.
            with open(self.path,'wb') as f:
                shutil.copyfileobj(r.raw, f)

            print('Image sucessfully downloaded: ',self.path)
        else:
            print('Image couldn\'t be retreived')
            return 
        
        image_abs_path = os.path.abspath(self.path)
        if os.path.isfile(image_abs_path) and image_abs_path.split('.')[-1].lower() in ['jpg', 'jpeg', 'png']:
                      
            # Load Image
            img0 = cv2.imread("/home/model-server/vlcsnap.png")

            image = cv2.resize(img0, (640, 640))
            image_bchw = np.transpose(np.expand_dims(image, 0), (0, 3, 1, 2))
            return img0, image_bchw
        else:
            print("Invalid image format.")
            return

    def inference(self, model_input):
        print("Performing PyTorch prediction")
        start_time = datetime.now()
        prediction = self.model([model_input], size=640) # batch of images
        end_time = datetime.now()

        return prediction, (end_time - start_time).total_seconds()

    def postprocess(self, inference_output):
        if inference_output is not None:
            prediction = inference_output[0]
            inference_time = inference_output[1]

            # Bytestring option
            serialized = pickle.dumps(prediction)
            # deserialized_a = pickle.loads(serialized)

            # JSON option
            # latin-1 maps byte n to unicode code point n
            serialized_as_json = json.dumps(pickle.dumps(prediction).decode('latin-1'))
            
            print(prediction)
            # print(serialized_as_json)

            # return [f"inference_time: {inference_time}s\nInference_summary: {log_string}\nraw_output:\n{raw_output}"]
            return serialized_as_json #"]#{log_string}\nraw_output:\n{raw_output}"]

    def handle(self, data, context):
        preprocessed_data = self.preprocess(data)
        if preprocessed_data:
            model_input = preprocessed_data
            inference_output = self.inference(model_input)
        return self.postprocess(inference_output)

_service = ModelHandler()

def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)
    
    if data is None:
        return None

    return _service.handle(data, context)

In [ ]:
handle()

In [4]:
%cd #yolov5
# !pip3 install onnx>=1.10.0
model = 'yolov5s6' #@param ["yolov5n6", "yolov5s6", "yolov5m6", "yolov5l6", "yolov5x6"]
input_width = 640 #@param {type:"slider", min:64, max:4096, step:64}    
input_height = 640 #@param {type:"slider", min:64, max:4096, step:64}
!ls
!python3 export.py --weights /home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.pt --img {input_height} {input_width} --include onnx --simplify

[Errno 2] No such file or directory: '#yolov5'
/home/martin/Projects/ongoing/ukw/model/yolov5/yolov5
benchmarks.py	 detect.py   __pycache__       runs	       val.py
CITATION.cff	 export.py   pyproject.toml    segment	       yolov5s.pt
classify	 hubconf.py  README.md	       train.py
CONTRIBUTING.md  LICENSE     README.zh-CN.md   tutorial.ipynb
data		 models      requirements.txt  utils


/home/martin/Projects/ongoing/ukw/model/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


export: data=data/coco128.yaml, weights=['/home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=True, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from /home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.pt with output shape (1, 25200, 7) (13.8 MB)

ONNX: starting export with onnx 1.15.0...
requirements: Ultralytics requirement ['onnx-simplifier>=0.4.1'] not found, attempting AutoUpdate...
/home/martin/.pyenv/pyenv.d/exec/pip-rehash/pip: /home/martin/Projects/ongoing/ukw/model/.venv/bin/pip: /home/martin/Project

In [5]:
import cv2
import torch
from PIL import Image
import time
from super_gradients.training.utils.media.image import load_image
import numpy as np


# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.onnx')
# # Images
# for f in 'zidane.jpg', 'bus.jpg':
#     torch.hub.download_url_to_file('https://ultralytics.com/images/' + f, f)  # download 2 images
# im1 = Image.open('zidane.jpg')  # PIL image
# im2 = cv2.imread('bus.jpg')[..., ::-1]  # OpenCV image (BGR to RGB)

# Inference
# Load and preprocess image for testing
image = np.asarray(Image.open("/home/martin/Desktop/wildfire.jpg"))
image = cv2.resize(image, (640, 640))
image_bchw = np.transpose(np.expand_dims(image, 0), (0, 3, 1, 2))

    

start = time.perf_counter()
results = model([image], size=640) # batch of images
end = time.perf_counter()
print(f"Time: {end-start} s")

# Results
results.print()  
results.save(save_dir='/home/martin/Desktop')  # or .show()

results.xyxy[0]  # im1 predictions (tensor)
results.pandas().xyxy[0]  # im1 predictions (pandas)
#      xmin    ymin    xmax   ymax  confidence  class    name
# 0  749.50   43.50  1148.0  704.5    0.874023      0  person
# 1  433.50  433.50   517.5  714.5    0.687988     27     tie
# 2  114.75  195.75  1095.0  708.0    0.624512      0  person
# 3  986.00  304.00  1028.0  420.0    0.286865     27     tie


The console stream is logged into /home/martin/sg_logs/console.log


[2024-05-06 13:59:42] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


[WARNING]No module named 'pycocotools'


I0506 13:59:48.125076 133696912667072 env_sanity_check.py:112] numpy==1.26.4 does not satisfy requirement numpy<=1.23
Using cache found in /home/martin/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirement ['pillow>=10.3.0'] not found, attempting AutoUpdate...
requirements: ❌ Command 'pip install --no-cache "pillow>=10.3.0" ' returned non-zero exit status 126.


/home/martin/.pyenv/pyenv.d/exec/pip-rehash/pip: /home/martin/Projects/ongoing/ukw/model/.venv/bin/pip: /home/martin/Projects/ukw/ukw/model/.venv/bin/python: bad interpreter: No such file or directory
YOLOv5 🚀 2024-5-4 Python-3.10.12 torch-2.2.1+cu121 CPU

Loading /home/martin/Projects/ongoing/ukw/model/yolov5_freeze_ds_fire_3_pre/best.onnx for ONNX Runtime inference...
Adding AutoShape... 


,xmin,ymin,xmax,ymax,confidence,class,name
0,413.728699,303.006348,457.353577,324.346130,0.745950,0,fire
1,520.301025,241.078064,581.076050,312.585632,0.625424,1,smoke
2,375.935272,214.437576,399.287506,240.502853,0.619318,0,fire
3,59.458191,353.438629,104.690796,406.029510,0.520865,1,smoke
4,291.413544,281.091492,318.496124,301.168823,0.519207,0,fire
5,414.060730,241.252884,434.034241,261.348602,0.413312,0,fire
6,123.702484,325.788879,269.906891,383.386658,0.394778,0,fire
7,227.961548,54.212837,249.512054,79.270470,0.371741,0,fire
8,357.098969,252.612656,400.373505,299.891907,0.369598,1,smoke
9,163.004089,337.698151,256.178314,376.231171,0.344392,0,fire


Time: 0.3468995900002483 s


image 1/1: 640x640 10 fires, 5 smokes
Speed: 5.3ms pre-process, 321.1ms inference, 18.1ms NMS per image at shape (1, 3, 640, 640)
Saved 1 image to /home/martin/Desktop3


In [26]:
results.pandas().xyxy[0].to_dict()

<bound method DataFrame.to_dict of           xmin        ymin        xmax        ymax  confidence  class   name
0   413.728699  303.006378  457.353577  324.346100    0.745950      0   fire
1   520.301086  241.078064  581.076111  312.585632    0.625423      1  smoke
2   375.935242  214.437576  399.287537  240.502853    0.619319      0   fire
3    59.458199  353.438629  104.690788  406.029510    0.520866      1  smoke
4   291.413544  281.091492  318.496124  301.168823    0.519207      0   fire
5   414.060699  241.252884  434.034210  261.348572    0.413311      0   fire
6   123.702484  325.788879  269.906891  383.386658    0.394778      0   fire
7   227.961563   54.212845  249.512039   79.270462    0.371742      0   fire
8   357.098969  252.612640  400.373505  299.891937    0.369598      1  smoke
9   163.004089  337.698151  256.178345  376.231171    0.344392      0   fire
10  107.903175  262.204987  198.418793  336.339081    0.336605      1  smoke
11  272.760071  306.216064  288.313660  3

In [27]:
flat_predictions = results.xyxyn
for (x_min, y_min, x_max, y_max, confidence, class_id) in flat_predictions[0]:
    class_id = int(class_id)
    print(f"Detected object with class_id={class_id}, confidence={confidence}, x_min={x_min}, y_min={y_min}, x_max={x_max}, y_max={y_max}")

Detected object with class_id=1, confidence=0.35458868741989136, x_min=0.5154963731765747, y_min=0.41392049193382263, x_max=0.5646795034408569, y_max=0.4997726380825043
Detected object with class_id=1, confidence=0.340872198343277, x_min=0.5772157907485962, y_min=0.46482688188552856, x_max=0.6342465281486511, y_max=0.5345455408096313
Detected object with class_id=1, confidence=0.2519384026527405, x_min=0.5353363752365112, y_min=0.43407759070396423, x_max=0.6365019679069519, y_max=0.5308430194854736


In [28]:
[i[0] for i in flat_predictions]

flat_predictions

[tensor([[0.51550, 0.41392, 0.56468, 0.49977, 0.35459, 1.00000],
         [0.57722, 0.46483, 0.63425, 0.53455, 0.34087, 1.00000],
         [0.53534, 0.43408, 0.63650, 0.53084, 0.25194, 1.00000]])]

In [31]:
import matplotlib.pyplot as plt
from super_gradients.training.datasets.datasets_conf import COCO_DETECTION_CLASSES_LIST
from super_gradients.training.utils.detection_utils import DetectionVisualization

def show_predictions_from_flat_format(image, predictions):
    [flat_predictions] = predictions

    image = image.copy()
    class_names = ['smoke', 'fire'] #COCO_DETECTION_CLASSES_LIST
    color_mapping = DetectionVisualization._generate_color_mapping(len(class_names))

    for (x1, y1, x2, y2, class_score, class_index) in flat_predictions:
        class_index = int(class_index)
        image = DetectionVisualization.draw_box_title(
                    image_np=image,
                    x1=int(x1),
                    y1=int(y1),
                    x2=int(x2),
                    y2=int(y2),
                    class_id=class_index,
                    class_names=class_names,
                    color_mapping=color_mapping,
                    box_thickness=1,
                    pred_conf=class_score,
                )

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.tight_layout()
    plt.show()

In [32]:
%matplotlib inline

image = load_image('/home/martin/Desktop/ukw/test_def.jpeg')
image = cv2.resize(image, (640, 640))
image_bchw = np.transpose(np.expand_dims(image, 0), (0, 3, 1, 2))

show_predictions_from_flat_format(image, flat_predictions)

TypeError: type Tensor doesn't define __round__ method

In [13]:
results

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 640x640 3 smokes
Speed: 7.3ms pre-process, 415.9ms inference, 15.9ms NMS per image at shape (1, 3, 640, 640)

In [11]:
class_names = ['smoke', 'fire']
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, yma


def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)
        width = xmax - xmin
        height = ymax - ymin

        class_name = class_names[int(labels[box_num])]

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=colors[class_names.index(class_name)],
            thickness=2
        )

        font_scale = min(1,max(3,int(w/500)))
        font_thickness = min(2, max(10,int(w/50)))

        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
        # Text width and height
        tw, th = cv2.getTextSize(
            class_name,
            0, fontScale=font_scale, thickness=font_thickness
        )[0]
        p2 = p1[0] + tw, p1[1] + -th - 10
        cv2.rectangle(
            image,
            p1, p2,
            color=colors[class_names.index(class_name)],
            thickness=-1,
        )
        cv2.putText(
            image,
            class_name,
            (xmin+1, ymin-10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            (255, 255, 255),
            font_thickness
        )
    return image

In [12]:
plot_box(image, np.array(results.xyxy[0][0][:4]).tolist(), class_names)

TypeError: 'float' object is not subscriptable

In [ ]:
import torch
from torchinfo import summary
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback



[2024-03-17 16:26:01] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-03-17 16:26:01] WARNING - __init__.py - Failed to import pytorch_quantization


The console stream is logged into /home/martin/sg_logs/console.log
[WARNING]No module named 'pycocotools'


[2024-03-17 16:26:02] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-03-17 16:26:02] WARNING - export.py - Failed to import pytorch_quantization
[2024-03-17 16:26:02] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [ ]:
dataset_params = {
    'data_dir':'/home/martin/Projects/ukw/ukw/data/wildfire-2',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': ['smoke']
}


In [ ]:

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

# clear_output()

[2024-03-17 16:27:17] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 4836/4836 [00:00<00:00, 12735.96it/s]
[2024-03-17 16:27:17] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 1379/1379 [00:00<00:00, 13768.54it/s]
[2024-03-17 16:27:17] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:   0%|          | 0/692 [00:00<?, ?it/s]

Indexing dataset annotations: 100%|██████████| 692/692 [00:00<00:00, 12855.71it/s]


In [ ]:

train_params = {
    # ENABLING SILENT MODE
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 10,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

/home/martin/Projects/ukw/ukw/model/.venv/lib/python3.10/site-packages/super_gradients/training/losses/ppyolo_loss.py:669: DeprecationWarning: A reg_max argument is not needed for PPYoloE loss anymore. It is deprecated since SG 3.6.0 and will be removed in the SG 3.8.0.You can safely omit this argument as it is not used anymore and we infer it automatically from model's outputs
  warnings.warn(


In [ ]:
import torch

model = torch.hub.load(
    "ultralytics/yolov5", "custom", "best.pt"
)  # load from PyTorch Hub

Using cache found in /home/martin/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-16 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
CHECKPOINT_DIR = 'checkpoints'

trainer = Trainer(experiment_name='yolonas_s', ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
# model: nn.Module,

trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

[2024-03-17 17:03:32] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20240317_170332_950849`
[2024-03-17 17:03:32] INFO - sg_trainer.py - Checkpoints directory: checkpoints/yolonas_s/RUN_20240317_170332_950849
[2024-03-17 17:03:32] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
/home/martin/Projects/ukw/ukw/model/.venv/lib/python3.10/site-packages/super_gradients/common/registry/registry.py:72: DeprecationWarning: Object name `linear_epoch_step` is now deprecated. Please replace it with `LinearEpochLRWarmup`.
  warnings.warn(f"Object name `{name}` is now deprecated. Please replace it with `{deprecated_names[name]}`.", DeprecationWarning)
[2024-03-17 17:03:32] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               0          (0 available on the machine)
    - Full dataset size:            4836       (len(train_set))
    - Batch size per GPU:           16     

The console stream is now moved to checkpoints/yolonas_s/RUN_20240317_170332_950849/console_Mar17_17_03_32.txt


ValueError: not enough values to unpack (expected 6, got 3)

Train epoch 0:   0%|          | 0/302 [00:02<?, ?it/s]
[2024-03-17 17:03:35] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process


In [ ]:
# Import the necessary module
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback


trainer.test(model=model,
            test_loader=test_data,
            test_metrics_list=DetectionMetrics_050(score_thres=0.45,
                                                   top_k_predictions=300,
                                                   num_cls=len(dataset_params['classes']),
                                                   normalize_targets=True,
                                                   post_prediction_callback=PPYoloEPostPredictionCallback(score_threshold=0.01,
                                                                                                          nms_top_k=1000,
                                                                                                          max_predictions=300,
                                                                                                          nms_threshold=0.7)
                                                  ))

Testing:   0%|          | 0/44 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 6)

Testing:   0%|          | 0/44 [00:02<?, ?it/s]
